In [1]:
import numpy as np
import pydot
from IPython.display import display, SVG, clear_output

from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.all import (Parser, StartMeshcat, DiagramBuilder,
                        MeshcatVisualizerCpp, JacobianWrtVariable,
                        MakeRenderEngineVtk, RenderEngineVtkParams)
from pydrake.geometry import (Box, Cylinder)
from pydrake.multibody.tree import (PrismaticJoint, UnitInertia, SpatialInertia, RevoluteJoint, FixedOffsetFrame, WeldJoint)
from manipulation.meshcat_cpp_utils import MeshcatJointSliders
from manipulation.scenarios import AddMultibodyTriad
meshcat = StartMeshcat()

Meshcat is now available at http://localhost:7000


In [5]:
# Build robot model in Drake
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.0)

# parameters
L0 = 0.8 #lenght of link 0
L1 = 0.5
L2 = 1

RGBA_Color0 = [0.5, 0.5, 0.5, 0.4]
RGBA_Color1 = [0, 0.5, 0.5, 0.4]
RGBA_Color2 = [0.9, 0, 0, 0.4]

my_model_instance = plant.AddModelInstance("my_robot")

#SpatialInertia(mass, reference point (wrt CoM), UnitInertia()): 6x6 matrix
default_inertia = SpatialInertia(1, [0, 0, L0/2], UnitInertia(1, 1, 1))  #This does not matter for kinematics

link_0 = plant.AddRigidBody("link_0", my_model_instance, default_inertia)

link_1 = plant.AddRigidBody("link_1", my_model_instance, default_inertia)

link_2 = plant.AddRigidBody("link_2", my_model_instance, default_inertia)

# plant.RegisterVisualGeometry(body, 
c =0.05
plant.RegisterVisualGeometry(
    link_0,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L0/2]),
    Cylinder(c, L0),
    "link_0",
    RGBA_Color0)
plant.RegisterVisualGeometry(
    link_1,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L1/2]),
    Box(c,c, L1),
    "link_1",
    RGBA_Color1)
plant.RegisterVisualGeometry(
    link_2,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L2/2]),
    Box(c,c, L2),
    "link_2",
    RGBA_Color2)

Joint1_Frame = plant.AddFrame(FixedOffsetFrame(
    link_0,
    RigidTransform(RollPitchYaw(0, np.pi/2, 0), [0, 0, L0])))

Joint2_Frame = plant.AddFrame(FixedOffsetFrame(
    link_1,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L1])))

Joint1 = plant.AddJoint(RevoluteJoint(
    name="Joint1", frame_on_parent=Joint1_Frame,
    frame_on_child=link_1.body_frame(), axis=[-1, 0, 0],
    pos_lower_limit=-3.14,
    pos_upper_limit=3.14,
    damping=0.0))
Joint2 = plant.AddJoint(RevoluteJoint(
    name="Joint2", frame_on_parent=Joint2_Frame,
    frame_on_child=link_2.body_frame(), axis=[-1, 0, 0],
    pos_lower_limit=-3.14,
    pos_upper_limit=3.14,
    damping=0.0))

plant.WeldFrames(
    frame_on_parent_P=plant.world_frame(),
    frame_on_child_C=link_0.body_frame(),
    X_PC=RigidTransform.Identity())


# add frames of interest
G = plant.AddFrame(FixedOffsetFrame('EndEffector',
    link_2,
    RigidTransform(RollPitchYaw(0, -np.pi/2, 0), [0, 0, L2])))

In [6]:
# Draw RGB frames for visualization
for body_name in ["link_0", "link_1", "EndEffector"]:
    AddMultibodyTriad(plant.GetFrameByName(body_name), scene_graph, 0.20, 0.008)



In [7]:
# Finalize and visualize
plant.Finalize()
renderer_name = "renderer"
scene_graph.AddRenderer(renderer_name, MakeRenderEngineVtk(RenderEngineVtkParams()))
meshcat.Delete()
meshcat_vis = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
diagram.Publish(diagram_context)

In [15]:
# Get world frame
theta1=np.pi/5
theta2=np.pi/4
plant.SetPositions(plant.GetMyContextFromRoot(diagram_context), my_model_instance, [theta1, theta2]) # theta1, theta2
diagram.Publish(diagram_context)
plant_context = plant.GetMyMutableContextFromRoot(diagram_context)
X_WG=G.CalcPoseInWorld(plant_context)
print(X_WG)


RigidTransform(
  R=RotationMatrix([
    [0.15643446504023095, -0.9876883405951378, -5.1653491612975294e-17],
    [0.9876883405951377, 0.15643446504023095, 6.047846824324981e-17],
    [-5.1653491612975294e-17, -6.047846824324981e-17, 1.0],
  ]),
  p=[0.5609429622277047, 1.2815809667413742, 0.8],
)


In [16]:
# now let's verify using PoE 

import modern_robotics as mr

# step 0: compute M 
M = mr.RpToTrans(np.eye(3), np.array([L1+L2, 0, L0]))
# step 1: compute all the screw axis
Sbar_2_0=mr.ScrewToAxis(np.array([L1,0,0]),np.array([0,0,1]),0)
Sbar_1_0=mr.ScrewToAxis(np.array([0,0,0]),np.array([0,0,1]),0)

def myPoE(theta1,theta2):
    SbarMatrix_2_0=mr.VecTose3(Sbar_2_0)
    SbarMatrix_1_0=mr.VecTose3(Sbar_1_0)
    return mr.MatrixExp6(SbarMatrix_1_0*theta1)@mr.MatrixExp6(SbarMatrix_2_0*theta2)@M


Tbs = myPoE(theta1,theta2)
print(Tbs)

[[ 0.15643447 -0.98768834  0.          0.56094296]
 [ 0.98768834  0.15643447  0.          1.28158097]
 [ 0.          0.          1.          0.8       ]
 [ 0.          0.          0.          1.        ]]


In [12]:
print(Sbar_2_0)

[ 0.   0.   1.   0.  -0.5  0. ]
